
# Smartvote – Clean Clustering Pipeline  
*(generated 2025-06-02)*

Dieses Notebook erstellt saubere Themen‑Cluster aus deutschsprachigen Fragen
und exportiert die Ergebnisse als Excel‑Datei.

**Highlights gegenüber der alten Version**

* nur die benötigten Spalten werden exportiert  
* keine verschachtelten `noise_noise_…`‑Labels mehr  
* Cluster mit weniger als 2 Fragen werden automatisch als *Noise* behandelt  
* entfernte Duplikate werden am Ende wieder ihrem Repräsentanten‑Cluster
  zugeordnet und mit ausgegeben  
* klarer, modularer Code – jede Zelle macht genau einen Schritt und gibt
  eine kurze Zusammenfassung aus  


In [ ]:

from pathlib import Path
import pandas as pd
import numpy as np

from sentence_transformers import SentenceTransformer, util
import hdbscan
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import DBSCAN
from nltk.corpus import stopwords
import nltk, sys, warnings, math

# ---------- Parameter ----------
DATA_PATH              = Path('df_de_final.xlsx')       # Eingabedatei
EXPORT_PATH            = Path('cluster_ergebnis.xlsx')  # Ausgabe‑Datei

SBERT_MODEL_NAME       = 'paraphrase-multilingual-MiniLM-L12-v2'

# Clustering
HDBSCAN_MIN_CLUSTER_SIZE = 2
HDBSCAN_MIN_SAMPLES      = None        # None = auto = min_cluster_size//2
MIN_FINAL_CLUSTER_SIZE   = 2           # alles <2 wird als Noise behandelt

# Duplikaterkennung
DUP_EPS = 0.05                         # DBSCAN‑Epsilon im TF‑IDF‑Raum

# Re‑Assignment
REASSIGN_SIM_THRESH = 0.70             # Schwelle, ab der Noise‑Fragen einem Cluster zugeordnet werden

# NLTK‐Stopwörter laden (nur 1× pro Session)
nltk.download('stopwords', quiet=True)
GERMAN_STOP = stopwords.words('german')

print('Parameter geladen.')


In [ ]:

# 1 ) Daten laden & Grundreinigung
df_all = pd.read_excel(DATA_PATH)
print(f'Geladene Zeilen: {len(df_all):,}')

df_all['frage_text_norm'] = (
    df_all['Frage_Text'].astype(str)
          .str.strip()
          .str.lower()
)

print('Norm‑Spalte fertig.')


In [ ]:

# 2 ) Duplikaterkennung mit TF‑IDF + DBSCAN
tfidf  = TfidfVectorizer(stop_words=GERMAN_STOP)
X_tfidf = tfidf.fit_transform(df_all['frage_text_norm'])

dbscan = DBSCAN(eps=DUP_EPS, min_samples=2, metric='cosine')
dupe_labels = dbscan.fit_predict(X_tfidf)

df_all['dupe_cluster']  = dupe_labels
df_all['dupe_is_main']  = df_all.groupby('dupe_cluster').cumcount() == 0

# --- Repräsentanten auswählen & eindeutige Fragen extrahieren ---
unique_mask = (df_all['dupe_cluster'] == -1) | (df_all['dupe_is_main'])
df_unique   = df_all[unique_mask].copy().reset_index(drop=True)

print(f'Eindeutige Fragen nach Duplikaterkennung: {len(df_unique):,}')


In [ ]:

# 3 ) SBERT‑Embeddings berechnen
model = SentenceTransformer(SBERT_MODEL_NAME)
embeddings = model.encode(
    df_unique['frage_text_norm'].tolist(),
    show_progress_bar=True,
    convert_to_tensor=True
)
print('Embeddings erstellt.')


In [ ]:

# 4 ) HDBSCAN‑Clustering
clusterer = hdbscan.HDBSCAN(
    min_cluster_size      = HDBSCAN_MIN_CLUSTER_SIZE,
    min_samples           = HDBSCAN_MIN_SAMPLES,
    metric                = 'euclidean',
    cluster_selection_method = 'eom'
)
cluster_labels = clusterer.fit_predict(embeddings.cpu().numpy())
df_unique['cluster'] = cluster_labels

print('Anzahl Cluster (exkl. Noise):',
      len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0))


In [ ]:

# 5 ) Ein‑Fragen‑Cluster als Noise behandeln
size_map = df_unique.groupby('cluster').size()
tiny_clusters = size_map[size_map < MIN_FINAL_CLUSTER_SIZE].index

df_unique.loc[df_unique['cluster'].isin(tiny_clusters), 'cluster'] = -1
print(f'Als Noise markiert (Grösse <{MIN_FINAL_CLUSTER_SIZE}): {len(tiny_clusters)} Cluster')


In [ ]:

# 6 ) Noise‑Punkte optional zuordnen
import torch
noise_idx = df_unique[df_unique['cluster'] == -1].index

if len(noise_idx):
    noise_emb = embeddings[noise_idx]
    clustered_mask = df_unique['cluster'] != -1
    clustered_emb  = embeddings[clustered_mask]

    for i, idx in enumerate(noise_idx):
        sims = util.cos_sim(noise_emb[i], clustered_emb)[0].cpu().numpy()
        best_idx   = sims.argmax()
        best_sim   = sims[best_idx]

        if best_sim >= REASSIGN_SIM_THRESH:
            best_cluster = df_unique.iloc[np.where(clustered_mask)[0][best_idx]]['cluster']
            # -------- sauberes noise‑Label ohne Verdopplung -----------
            if isinstance(best_cluster, str) and best_cluster.startswith('noise_'):
                base = best_cluster.split('noise_')[-1]
                df_unique.at[idx, 'cluster'] = f'noise_{base}'
            else:
                df_unique.at[idx, 'cluster'] = f'noise_{best_cluster}'

print('Noise‑Reassignment abgeschlossen.')


In [ ]:

# 7 ) Hauptfrage & Ähnlichkeit
df_unique['ist_hauptfrage']              = False
df_unique['ähnlichkeit_zur_hauptfrage']  = np.nan

final_emb = model.encode(df_unique['frage_text_norm'].tolist(),
                         convert_to_tensor=True)

for cl in df_unique['cluster'].unique():
    cl_mask = df_unique['cluster'] == cl
    idxs    = np.where(cl_mask)[0]
    if not len(idxs):
        continue
    centroid = final_emb[idxs].mean(dim=0, keepdim=True)
    sims     = util.cos_sim(centroid, final_emb[idxs])[0].cpu().numpy()

    best_local_idx = idxs[sims.argmax()]
    df_unique.at[best_local_idx, 'ist_hauptfrage'] = True
    df_unique.loc[cl_mask, 'ähnlichkeit_zur_hauptfrage'] = sims

print('Hauptfragen markiert & Ähnlichkeiten berechnet.')


In [ ]:

# 8 ) Cluster auf alle Duplikats‑Zeilen übertragen
# 8a) Cluster‑Info von df_unique an df_all mergen
df_all = df_all.merge(
    df_unique[['ID_gesamt', 'cluster']],
    on='ID_gesamt',
    how='left'
)

# 8b) Fehlende Cluster über Repräsentanten‑Map auffüllen
rep_map = (df_all[df_all['dupe_is_main']]
           .set_index('dupe_cluster')['cluster'])

def fill_cluster(row):
    if pd.isna(row['cluster']):
        return rep_map.get(row['dupe_cluster'], -1)
    return row['cluster']

df_all['cluster'] = df_all.apply(fill_cluster, axis=1)

print('Cluster auf alle Zeilen übertragen.')


In [ ]:

# 9 ) Finales DataFrame + Export
export_cols = ['cluster',
               'ist_hauptfrage',
               'ähnlichkeit_zur_hauptfrage',
               'Frage_Text',
               'ID_gesamt']

# Flag & Ähnlichkeit für Nicht‑Repräsentanten ergänzen (optional)
# -> Duplikate erhalten dieselbe Ähnlichkeit wie ihr Repräsentant
sim_map = (df_unique
           .set_index('ID_gesamt')['ähnlichkeit_zur_hauptfrage'])
df_all['ähnlichkeit_zur_hauptfrage'] = df_all['ID_gesamt'].map(sim_map)

# Hauptfrage‑Flag für Duplikate = False
df_all['ist_hauptfrage'] = df_all['ID_gesamt'].isin(
    df_unique[df_unique['ist_hauptfrage']]['ID_gesamt']
)

df_export = df_all[export_cols].copy()

# sinnvolle Sortierung
cluster_sizes = df_export.groupby('cluster').size().sort_values(ascending=False)
df_export['cluster'] = pd.Categorical(df_export['cluster'],
                                      categories=cluster_sizes.index,
                                      ordered=True)
df_export.sort_values(['cluster', 'ist_hauptfrage',
                       'ähnlichkeit_zur_hauptfrage'],
                      ascending=[True, False, False],
                      inplace=True)

# --- Excel ---
df_export.to_excel(EXPORT_PATH, index=False)
print(f'✓ Export fertig: {EXPORT_PATH}')
print('\nVorschau:')
display(df_export.head())
